# Assignment 4

Before working on this assignment please read these instructions fully. In the submission area, you will notice that you can click the link to **Preview the Grading** for each step of the assignment. This is the criteria that will be used for peer grading. Please familiarize yourself with the criteria before beginning the assignment.

This assignment requires that you find **at least two datasets** on the web which are related, and that you visualize these datasets to answer the assignment question. You are free to utilize datasets with any location or domain, the usage of **Ann Arbor sports and athletics** datasets in the example is just a suggestion.

You are welcome to choose datasets at your discretion, but keep in mind **they will be shared with your peers**, so choose appropriate datasets. Sensitive, confidential, illicit, and proprietary materials are not good choices for datasets for this assignment. You are welcome to upload datasets of your own as well, and link to them using a third party repository such as github, pastebin, etc. Please be aware of the Coursera terms of service with respect to intellectual property.

Also, you are welcome to preserve data in its original language, but for the purposes of grading you should provide english translations. You are welcome to provide multiple visuals in different languages if you would like!

As this assignment is for the whole course, you must incorporate principles discussed in the first week, such as having as high data-ink ratio (Tufte) and aligning with Cairo’s principles of truth, beauty, function, and insight.

Here are the assignment instructions:

 * You must state a question you are seeking to answer with your visualizations.
 * You must provide at least two links to available datasets. These could be links to files such as CSV or Excel files, or links to websites which might have data in tabular form, such as Wikipedia pages.
 * You must upload an image which addresses the research question you stated. In addition to addressing the question, this visual should follow Cairo's principles of truthfulness, functionality, beauty, and insightfulness.
 * You must contribute a short (1-2 paragraph) written justification of how your visualization addresses your stated research question.

## Tips
* Wikipedia is an excellent source of data, and I strongly encourage you to explore it for new data sources.
* Many governments run open data initiatives at the city, region, and country levels, and these are wonderful resources for localized data sources.
* Several international agencies, such as the [United Nations](http://data.un.org/), the [World Bank](http://data.worldbank.org/), the [Global Open Data Index](http://index.okfn.org/place/) are other great places to look for data.
* This assignment requires you to convert and clean datafiles. Check out the discussion forums for tips on how to do this from various sources, and share your successes with your fellow students!

## Example
Looking for an example? Here's what our course assistant put together as an example! [Example Solution File](./Assignment4_example.pdf)

In [1]:
# ============== Importing the necessary Libraries =================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact


def cleaned_df():
    #================ Dealing with the GDP df ============================
    GDP_df = pd.read_csv('./GDP.csv', encoding='Latin-1', skiprows=[0])
    
    # GDP per capita data extraction
    GDP = GDP_df[(GDP_df['Series']=='GDP per capita (US dollars)')]
    GDP.rename(columns={'Unnamed: 1':'Country', 'Value':'GDP per capita (US dollars)', 'Region/Country/Area':'Code'}, inplace = True)
    GDP.drop(columns=['Footnotes','Source', 'Series'], inplace=True)

    # GDP in current prices data extraction
    GDP2 = GDP_df[(GDP_df['Series']=='GDP in current prices (millions of US dollars)')]     # selecting relvent rows from the df
    GDP2.rename(columns={'Unnamed: 1':'Country', 'Value':'GDP in current prices (millions of US dollars)', 'Region/Country/Area':'Code'}, inplace = True)
    GDP2.drop(columns=['Footnotes','Source', 'Series'], inplace=True)  # removing irrelevant cols

    GDP['GDP per capita (US dollars)'] = GDP['GDP per capita (US dollars)'].str.replace(',','').astype(float)
    GDP2['GDP in current prices (millions of US dollars)'] = GDP2['GDP in current prices (millions of US dollars)'].str.replace(',','').astype(float)
    # GDP2
    
    # Merging the 2 GDP df since they have similar rows except their GDP values 
    GDP_df = pd.merge(GDP, GDP2, on = ['Country', 'Year', 'Code'], how = 'inner')
    # GDP_df
    
    #  Filtering out non-countries, i.e continents and sub-continents
    GDP_Reg = GDP_df[(GDP_df['Code'].isin([1,2,15,202,14,17,18,11,19,21,419,29,
                          13, 5, 142, 143, 30, 35, 34, 145, 150,
                          151, 154, 39, 155, 9, 53, 54, 57, 61]))] # GDP based on the region, continent or sub-continent

    # GDP by country
    GDP_by_country = GDP_df[~(GDP_df['Code'].isin([1,2,15,202,14,17,18,11,19,21,419,29,
                          13, 5, 142, 143, 30, 35, 34, 145, 150,
                          151, 154, 39, 155, 9, 53, 54, 57, 61]))] # Assigned the GDP values to the countries
    
    GDP_df2 = GDP_by_country.groupby(['Country', 'Year']).max() # sort the df using the country and year (Multi-Index)
    # GDP_df2
    
    
    
    # ================= Handling the CO2 emssions data ===========================
    CO2 = pd.read_csv('./CO2 emission.csv', encoding='Latin-1', skiprows=[0])
    # CO2
    
    # Looking at the CO2 emmsions per capita to make it tally with the GDP data
    CO2_perCapita = CO2[~(CO2['Series'] == 'Emissions (thousand metric tons of carbon dioxide)')]
    # CO2_perCapita
   
    # Cleaning the CO2 data

    CO2_perCapita.rename(columns={'Region/Country/Area':'Code','Unnamed: 1':'Country',
                       'Value':'Emissions per capita (Mt CO2)'}, inplace = True)

    CO2_perCapita.drop(columns=['Footnotes', 'Source', 'Series'], inplace = True)
    CO2_perCapita['Emissions per capita (Mt CO2)'] = CO2_perCapita['Emissions per capita (Mt CO2)'].astype(float) # convert to float since intial dtype is str
    
    CO2_df = CO2_perCapita.groupby(['Country', 'Year']).max() # Group CO2 df
    
    # Cumm sum of each country's emission
    cum_CO2_emmisions = CO2_df.groupby('Country')['Emissions per capita (Mt CO2)'].cumsum() # this cumsum is based on the country
    CO2_df['Cummulative CO2 Emmisions per capita (MtCO2)'] = cum_CO2_emmisions.values # new col to store the cumsum

    
    # ==================== Merging the 2 df (i.e. CO2 and GDP) ===========================
    df = pd.merge(CO2_df, GDP_df2, how='inner', left_index=True, right_index=True)
    df.drop(columns=['Code_x', 'Code_y'], inplace = True)
    
    
    
    return df
